In [1]:
# 라이브러리 임포트
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, f1_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

In [4]:
# 데이터 임포트

# 학습 데이터 불러오기 및 병합
train = pd.merge(
    pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train.csv'),
    pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train_labels.csv')[target],
    left_index=True,
    right_index=True
)

# 테스트 데이터 불러오기 (애초부터 떨어져 새로 수집된걸 추천!)
test = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/test.csv')


In [5]:
# 목표 변수 설정
target = 'vacc_h1n1_f'  # 목표 변수로 H1N1 백신 접종 여부를 설정합니다.
# target = 'vacc_seas_f'  # 다른 목표 변수로 계절 독감 백신 접종 여부도 사용 가능합니다.



# 데이터전처리
- 데이터 컬럼에 대한 철저한 이해! EDA
- 바차트, 선차트, 파이차트, 테이블, 히스토그램 기타등등

In [8]:
# 타겟과 특성 분리 
X  = train.drop(columns=[target])
y  = train[target]

## 데이터 클랜징
- 결측지 처리 ( 제거 / 대체 )
- 범주형 데이터 처리 ( 원핫 / 레이블 )


In [9]:
def preprocess_data(data):
    #수치형데이터 처리
    numeric = data.select_dtypes(include=['number'])
    numeric = numeric.fillna(numeric.mean())
    # 범주형데이터 처리
    # 예시)특수문자제거
    category = data.select_dtypes(include=['object'])
    category = category.fillna(category.mode().iloc[0])
    # 예시)특수문자제거
    category = pd.get_dummies(category) #get_dummies() 원핫인코딩 적용 매소드
    return pd.concat([numeric, category], axis=1)

In [10]:
X = preprocess_data(X)
test = preprocess_data(test )

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 훈련 / 예측

In [12]:
# 모델 설정
model = DecisionTreeClassifier(random_state=42)
# 모델 훈련
model.fit(X_train, y_train)
# 예측
y_pred = model.predict(X_val)

In [13]:
# 정확도
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7548333531016487


In [14]:
# confusion matrix
print(confusion_matrix(y_val, y_pred))

[[5310 1075]
 [ 992 1054]]


In [15]:
# 정밀도, 재현율, f1 score : classification_report()
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.83      0.84      6385
           1       0.50      0.52      0.50      2046

    accuracy                           0.75      8431
   macro avg       0.67      0.67      0.67      8431
weighted avg       0.76      0.75      0.76      8431



# Test data 훈련
- 이성능이 좋아야지, 일반화해서 사용가능!

In [16]:
test_pred = model.predict(test)

In [17]:
results = pd.DataFrame({
    'vacc_h1n1_f': test_pred
})

In [18]:
print(results.value_counts())

vacc_h1n1_f
0              21182
1               6922
Name: count, dtype: int64


# 결론
DecisionTreeClassifier 모델에서 75%의 정확도를 보이고 ,   
0 (백신미접종) : 정밀도 0.84 , 재현율 0.94 , f1 score : 0.84   
1(백신 접종) : 정밀도 0.50     재현율 0.52  f1 score    0.50   

모델 측면에서 볼때 백신 미접종자를 예측하는데 더 좋은 성능을 보이고, 백신 접종자를 식별하는데 어려움  

# 추후 시도 (회고)
- 데이터 보강
- 클래스 가중치 조정
- 더 복잡한 모델 적용
- feature egeernig 적용


